<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/Experimentos/Con%20Feedback/Experimento_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #@title RATE 15, 11 MARK 2
__author__ = 'yihanjiang'
# Adding the *Receiver Encoding*
import argparse
import math
import random
import torch
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import files
import csv
import locale
import pandas as pd
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-init_nw_weight', type=str, default='default')
    parser.add_argument('-code_rate', type=int, default=3)
    parser.add_argument('-learning_rate', type = float, default=0.00085)
    parser.add_argument('-batch_size', type=int, default=100)
    parser.add_argument('-num_epoch', type=int, default=10)
    parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
    parser.add_argument('-block_len', type=int, default=12)
    parser.add_argument('-num_block', type=int, default=100000)
    parser.add_argument('-enc_num_layer', type=int, default=2)
    parser.add_argument('-dec_num_layer', type=int, default=2)
    parser.add_argument('-fb_num_layer',  type=int, default=2)
    parser.add_argument('-enc_num_unit',  type=int, default=100)
    parser.add_argument('-dec_num_unit',  type=int, default=100)
    parser.add_argument('-fb_num_unit',   type=int, default=100)
    parser.add_argument('-q_train', type=float, default=0.01, help='Valor de probabilidad q de entrenamiento')
    parser.add_argument('-q_train_fb', type=float, default=0.0000001, help='Valor de probabilidad q de entrenamiento para feedback')
    parser.add_argument('-q_test_start', type=float, default=0.15, help='Inicio de valores de q para pruebas')
    parser.add_argument('-q_test_end', type=float, default=0.0001, help='Fin de valores de q para pruebas')
    parser.add_argument('-q_points', type=int, default=20, help='Número de puntos de probabilidad q')
    parser.add_argument('-channel_mode', choices=['normalize', 'lazy_normalize', 'tanh'], default='lazy_normalize')
    parser.add_argument('-enc_act', choices=['tanh', 'selu', 'relu', 'elu', 'sigmoid'], default='tanh')
    parser.add_argument('--zero_padding', action='store_true', default=False, help='enable zero padding')
    parser.add_argument("-f", "--file", required=False)
    args = parser.parse_args()
    return args
#############################
#                           #
#    █████████   ██████████ #
#   ███░░░░░███ ░░███░░░░░█ #
#  ░███    ░███  ░███  █ ░  #
#  ░███████████  ░██████    #
#  ░███░░░░░███  ░███░░█    #
#  ░███    ░███  ░███ ░   █ #
#  █████   █████ ██████████ #
# ░░░░░   ░░░░░ ░░░░░░░░░░  #
#                           #
#############################
class AE(torch.nn.Module):
    def __init__(self, args):
        super(AE, self).__init__()
        self.args             = args
        # Encoder
        self.enc_p2_rnn_fwd   = torch.nn.GRU(7, args.enc_num_unit,
                                           num_layers=args.enc_num_layer, bias=True, batch_first=True,
                                           dropout=0, bidirectional=False)
        self.enc_p2_linear    = torch.nn.Linear(args.enc_num_unit, 1)
        # Decoder
        '''
        self.dec_rnn           = torch.nn.GRU(4,  args.dec_num_unit,
                                           num_layers=2, bias=True, batch_first=True,
                                           dropout=0, bidirectional=True)
        self.dec_output        = torch.nn.Linear((2)*args.dec_num_unit, 3)
        '''

        self.dec_rnn           = torch.nn.GRU(16,  args.dec_num_unit,
                                           num_layers=2, bias=True, batch_first=True,
                                           dropout=0, bidirectional=True)
        self.dec_output        = torch.nn.Linear((2)*args.dec_num_unit, 12)

    def enc_act(self, inputs):
        if self.enc_act == 'tanh':
            return F.tanh(inputs)
        elif self.enc_act == 'elu':
            return F.elu(inputs)
        elif self.enc_act == 'relu':
            return F.relu(inputs)
        elif self.enc_act == 'selu':
            return F.selu(inputs)
        elif self.enc_act == 'sigmoid':
            return F.sigmoid(inputs)
        else:
            return F.tanh(inputs)
    def forward(self, input, fwd_noise, fb_noise, fwd_noise_cod, fb_noise_cod):
        x_p1      = input
        x_p1_norm = 2*x_p1-1
        p1_code = x_p1
        if self.args.zero_padding:
            p1_rec  = torch.logical_xor(x_p1, fwd_noise[:,:,0].view(self.args.batch_size, self.args.block_len+1, 1) ).float()
            p1_fb   = torch.logical_xor(p1_rec , fb_noise[:,:, 0].view(self.args.batch_size, self.args.block_len+1, 1) ).float()
        else:
            p1_rec  = torch.logical_xor(x_p1, fwd_noise[:,:,0].view(self.args.batch_size, self.args.block_len, 1) ).float()
            p1_fb   = torch.logical_xor(p1_rec , fb_noise[:,:, 0].view(self.args.batch_size, self.args.block_len, 1) ).float()

        p1_fb_norm = 2*p1_fb - 1

        for idx in range(0, input.shape[1], 3):
            if idx == 0:
                #print('xp1 shape = ', x_p1_norm.shape)
                #print('p1_fb_norm = ', p1_fb_norm.shape)
                #print((x_p1_norm[:,idx:idx+3,:]).shape)
                input_tmp        = torch.cat([x_p1_norm[:,idx:idx+3,:].view(self.args.batch_size, 3, 1),
                                              p1_fb_norm[:,idx:idx+3,:].view(self.args.batch_size, 3, 1),
                                              torch.zeros((self.args.batch_size, 1, 1)).to(device)], dim=1)
                x_p1_norm_horizontal = input_tmp.permute(0, 2, 1)
                x_fwd_p2, h_tmp  = self.enc_p2_rnn_fwd(x_p1_norm_horizontal)
                x_tmp_p2         = self.enc_act(self.enc_p2_linear(x_fwd_p2))
                x_p2_history     = x_tmp_p2
            else:
                input_tmp        = torch.cat([x_p1_norm[:,idx:idx+3,:].view(self.args.batch_size, 3, 1),
                                              p1_fb_norm[:,idx:idx+3,:].view(self.args.batch_size, 3, 1),
                                              fb_tmp.view((self.args.batch_size, 1, 1))], dim=1)
                x_p1_norm_horizontal = input_tmp.permute(0, 2, 1)
                x_tmp_p2, h_tmp  = self.enc_p2_rnn_fwd(x_p1_norm_horizontal, h_tmp)
                x_tmp_p2         = self.enc_act(self.enc_p2_linear(x_tmp_p2))
                x_p2_history     = x_tmp_p2

            x_tmp_p2  = torch.heaviside(x_tmp_p2 , torch.zeros(x_tmp_p2.shape, device=x_tmp_p2.device)).float()

            x_p2_rec    = torch.logical_xor(x_tmp_p2,  fwd_noise_cod[:,idx,:].view(self.args.batch_size, 1, 1)).float()
            x_p2_fb     = torch.logical_xor(x_p2_rec,  fb_noise_cod[:,idx,:].view(self.args.batch_size, 1, 1)).float()

            fb_tmp      = 2*x_p2_fb-1

            if idx == 0:
                p2_code= x_tmp_p2
                p2_rec = x_p2_rec
                p2_fb  = x_p2_fb
            else:
                p2_code = torch.cat([p2_code,x_tmp_p2 ], dim = 1)
                p2_rec = torch.cat([p2_rec,x_p2_rec ], dim = 1)
                p2_fb  = torch.cat([p2_fb, x_p2_fb],   dim = 1)


        #print('p1_rec = ', p1_rec.shape)
        #print('p2_rec = ', p2_rec.shape)

        dec_input = torch.cat([p1_rec,p2_rec], dim=1)
        dec_input_horizontal = dec_input.permute(0, 2, 1)
        #print('dec_input = ', dec_input_horizontal.shape)
        inputs_dec = 2*dec_input-1
        x_dec, hdec_tmp  = self.dec_rnn(dec_input_horizontal.float())
        #x_dec, hdec_tmp  = self.dec_rnn(inputs_dec.float() )
        x_dec     = F.sigmoid(self.dec_output(x_dec))
        return x_dec
####################
#                  #
#  ██████   ██████ #
# ░░██████ ██████  #
#  ░███░█████░███  #
#  ░███░░███ ░███  #
#  ░███ ░░░  ░███  #
#  ░███      ░███  #
#  █████     █████ #
# ░░░░░     ░░░░░  #
#                  #
####################
args = get_args()
print(args)
def errors_ber(y_true, y_pred):
    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])

    myOtherTensor = torch.not_equal(t1, t2).float()
    k = sum(sum(myOtherTensor))/(myOtherTensor.shape[0]*myOtherTensor.shape[1])
    return k
def errors_bler(y_true, y_pred):

    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])

    decoded_bits = t1
    X_test       = t2
    tp0 = (abs(decoded_bits-X_test)).reshape([X_test.shape[0],X_test.shape[1]])
    #tp0 = tp0.numpy()
    bler_err_rate = sum(torch.sum(tp0,axis=1)>0)*1.0/(X_test.shape[0])
    return bler_err_rate
def ber_pos(y_true, y_pred):
    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])
    dif = torch.not_equal(t1, t2).float()
    suma_dif = (torch.sum(dif, dim=0))
    promedio_errorxbit = suma_dif/args.batch_size
    return promedio_errorxbit
def descargar_datos(BER_FINAL, args, model):
    ber_par = [BER_FINAL[i] for i in range(len(BER_FINAL)) if i % 2 == 0]
    ber_impar = [BER_FINAL[i] for i in range(len(BER_FINAL)) if i % 2 != 0]
    column_names = ['init_nw_weight', 'code_rate', 'learning_rate', 'batch_size', 'num_epoch', 'no_cuda',
                    'block_len', 'num_block', 'enc_num_layer', 'dec_num_layer', 'fb_num_layer', 'enc_num_unit',
                    'dec_num_unit', 'fb_num_unit', 'q_train', 'q_train_fb', 'q_test_start', 'q_test_end',
                    'q_points', 'channel_mode', 'enc_act', 'zero_padding', 'file', 'BER_Par', 'BER_Impar', 'Model_Description']
    csv_file_path = '/content/datos.csv'
    model_description = str(model)
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=column_names)
        writer.writeheader()
        args_dict = vars(args)
        writer.writerow(args_dict)
        writer.writerow({'Model_Description': model_description})
    csv_df = pd.read_csv(csv_file_path)
    data = {'Probabilidad Q': ber_par, 'BER': ber_impar}
    ber_df = pd.DataFrame(data)
    excel_file_path = '/content/datos_con_args.xlsx'
    ber_df.T.to_excel(excel_file_path, index=True)
    excel_df = pd.read_excel(excel_file_path, index_col=0)
    df_concatenado = pd.concat([csv_df.T, excel_df])
    concatenated_file_path = '/content/Resultados.xlsx'
    df_concatenado.to_excel(concatenated_file_path, index=True)
    files.download(concatenated_file_path)
    return concatenated_file_path
def guardar_csv1(data, filename):
    filepath = filename  # Ajusta la ruta según dónde quieras guardar el archivo
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Q values'] + [f'BER_{i+1}' for i in range(data.shape[1]-1)])  # Encabezados para cada BER
        for row in data:
            writer.writerow(row)
    print(f"Archivo '{filename}' guardado correctamente.")
    files.download(filepath)

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx in range(int(num_train_block/args.batch_size)):
        if args.zero_padding:
            X_train    = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
            X_train    = torch.cat([X_train, torch.zeros(args.batch_size, 1, 1)], dim=1)
            fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len+1, args.code_rate), my_q_train))
            fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len+1, args.code_rate), my_q_train_fb))
        else:
            X_train    = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
            fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train))
            fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))
            fwd_noise_cod =  torch.bernoulli(torch.full((args.batch_size, args.block_len, 1), my_q_train))
            fb_noise_cod =   torch.bernoulli(torch.full((args.batch_size, args.block_len, 1), my_q_train_fb))

        X_train, fwd_noise, fb_noise, fwd_noise_cod, fb_noise_cod = X_train.to(device), fwd_noise.to(device), fb_noise.to(device), fwd_noise_cod.to(device), fb_noise_cod.to(device)
        optimizer.zero_grad()
        output = model(X_train, fwd_noise, fb_noise, fwd_noise_cod, fb_noise_cod)
        output_horizontal = output.permute(0, 2, 1)
        loss = F.binary_cross_entropy(output_horizontal, X_train)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} Loss: {:.6f}'.format(epoch, batch_idx, num_train_block/args.batch_size, loss.item()))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss /(num_train_block/args.batch_size)) )
def test(qs):
  model.eval()
  torch.manual_seed(random.randint(0,1000))
  print('Qs', qs)
  num_train_block =  args.num_block
  errorxbit_T = torch.zeros(1, args.block_len, 1)
  BER_FINAL = []
  num_test_batch = int(num_train_block / (args.batch_size * test_ratio))
  ber_q = np.zeros((len(qs), num_test_batch))
  for j,q in enumerate(qs):
    test_ber, test_bler = .0, .0
    with torch.no_grad():
      for batch_idx in range(num_test_batch):
        if args.zero_padding:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          X_test = torch.cat([X_test, torch.zeros(args.batch_size, 1, 1)], dim=1)
          fwd_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len + 1, args.code_rate), q))
          fb_noise = torch.zeros((args.batch_size, args.block_len + 1, args.code_rate))
        else:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), q))
          fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))
          fwd_noise_cod =  torch.bernoulli(torch.full((args.batch_size, args.block_len, 1), my_q_train))
          fb_noise_cod =   torch.bernoulli(torch.full((args.batch_size, args.block_len, 1), my_q_train_fb))
      X_test, fwd_noise, fb_noise, fwd_noise_cod, fb_noise_cod = X_test.to(device), fwd_noise.to(device), fb_noise.to(device), fwd_noise_cod.to(device), fb_noise_cod.to(device)
      X_hat_test = model(X_test, fwd_noise, fb_noise, fwd_noise_cod, fb_noise_cod).to(device)
      X_hat_test_horizontal = X_hat_test.permute(0, 2, 1)
      test_ber += errors_ber(X_hat_test_horizontal, X_test)
      test_bler += errors_bler(X_hat_test_horizontal, X_test)
      ber = errors_ber(X_hat_test_horizontal, X_test)
      ber_q[j][batch_idx] = ber
    test_ber  /= 1.0*num_test_batch
    test_bler /= 1.0*num_test_batch
    print('Test Q',round(q,3) ,'with ber ', float(test_ber), 'with bler', float(test_bler))
  return ber_q

identity = str(np.random.random())[2:8]
print('[ID]', identity)

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    model = AE(args).to(device)
else:
    model = AE(args)

print(model)

if args.init_nw_weight == 'default':
    pass
else:
    model = torch.load(args.init_nw_weight)
    model.args = args

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
test_ratio = 1
num_train_block, num_test_block = args.num_block, args.num_block/test_ratio
my_q_train = args.q_train
my_q_train_fb = args.q_train_fb

print('Traning Q is', my_q_train)

################
#              #
#  ███████████ #
# ░█░░░███░░░█ #
# ░   ░███  ░  #
#     ░███     #
#     ░███     #
#     ░███     #
#     █████    #
#    ░░░░░     #
#              #
################

#PATH='torch_model_791480.pt'
#model=torch.load(PATH)

q_interval = (args.q_test_end - args.q_test_start)* 1.0 /  (args.q_points-1)
qs = [q_interval* item + args.q_test_start for item in range(args.q_points)]
for epoch in tqdm(range(1, args.num_epoch + 1)):
    train(epoch)
    ber_q = test(qs)

qs1 = np.linspace(0.15, 0.0001, 20)
arrays_q_AE = []
num_epoch_test = 50
progreso_tanda = tqdm(total=num_epoch_test, desc=f"Progreso")
for _ in range(num_epoch_test):
  ber_q_AE = test(qs)
  arrays_q_AE.append(ber_q_AE)
  progreso_tanda.update(1)

AE_stack = np.hstack(arrays_q_AE)
qs_reshaped = qs1.reshape(20, 1)
AE_data = np.hstack((qs_reshaped, AE_stack))
AE_data_transposed = AE_data.T
guardar_csv1(AE_data_transposed, "CON FB-AE_15_11.csv")
print(AE_data_transposed.shape)

print("                                          ███████╗    ██╗    ███╗   ██╗")
print("                                          ██╔════╝    ██║    ████╗  ██║")
print("                                          █████╗      ██║    ██╔██╗ ██║")
print("                                          ██╔══╝      ██║    ██║╚██╗██║")
print("                                          ██║         ██║    ██║ ╚████║")
print("                                          ╚═╝         ╚═╝    ╚═╝  ╚═══╝")
#torch.save(model, '/content/drive/MyDrive/Deepcode_pytorch/tmp/torch_model_'+identity+'.pt')
#print('saved model', '/content/drive/MyDrive/Deepcode_pytorch/tmp/torch_model_'+identity+'.pt')